# Script to demodulate photometry experiment collected with Axon digidata 

In [2]:
## additional packages needed to run
from importlib import reload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import sys 
import neo 
from neo import io
import pandas as pd
import numpy as np
import yaml
import sys 
import photometry_3
get_ipython().magic('matplotlib qt')
#%matplotlib qt5
import matplotlib.pyplot as plt

In [4]:
## function for plotting graphs from stored data 
def plot_multiple_sweeps_from_class(class_name, df_to_load, sweeps_list, channels_list, window, wspace=0.1, hspace=0.1, 
                                    X_axis_column=None, X_label=None, Y_labels=None, _inline=False):
    """Inputs: 
    window (tuple of time values)""" 
    if sweeps_list == 'all':
        sweeps_to_plot = sorted(list(set(df_to_load.index.get_level_values(0))))
    else:
        sweeps_to_plot = sweeps_list
    windowed_start = int(window[0]*class_name.Fs)
    windowed_end = len(df_to_load.loc[sweeps_to_plot[0]][str(channels_list[0])]) - int(window[1]*class_name.Fs)
    if _inline == False:
        %matplotlib qt5
    else: 
        %matplotlib inline
    plt.subplots_adjust(wspace, hspace)
    
    
    # sweeps are plotted by column
    column_num = int(len(sweeps_to_plot))
    # channels are plotted by row
    row_num = int(len(channels_list))
    subplot_index = 1
    for channel_to_plot in range(len(channels_list)):
        for sweep in range(len(sweeps_to_plot)):
            plt.subplot(row_num, column_num, subplot_index)
            if X_axis_column:
                plt.plot(df_to_load.loc[str(sweeps_to_plot[sweep])][str(X_axis_column)].as_matrix()[windowed_start:windowed_end],
                        df_to_load.loc[str(sweeps_to_plot[sweep])][str(channels_list[channel_to_plot])].as_matrix()[windowed_start:windowed_end])
            else:
                plt.plot(df_to_load.loc[str(sweeps_to_plot[sweep])][str(channels_list[channel_to_plot])].as_matrix()[windowed_start:windowed_end])
            if X_label:
                plt.xlabel(X_label)
            if Y_labels:
                plt.ylabel(str(Y_labels[subplot_index]))
            else:
                plt.ylabel(str(channels_list[channel_to_plot]))
            subplot_index += 1
                           
    plt.show()
    return()
    

## input .abf file to load 

In [5]:
#input path to file 
file = '/Users/johnmarshall/Documents/Analysis/PhotometryData/November2018/11152018/18n15000.abf'
file_to_load = file
fname_for_saving = file.rstrip('.abf')
#use True or False to indicate if the experiment has rotary endocder data or not
#if True, also pass list with encoder channels
#need to add a column that has time according to run not just per sweep 
_recording = photometry_3.raw_photometry_file(file_to_load, .1, False)

In [6]:
_recording.class_df

channel_0  channel_1  channel_2  channel_3      time
sweep                                                                 
sweep001 0         4.936523  -0.000916  -0.000305   0.043640    0.0000
         1         4.933167  -0.000610  -0.000305   0.045471    0.0001
         2         4.936829   0.000305   0.000000   0.036011    0.0002
         3         4.938660   0.000610   0.000305   0.029602    0.0003
         4         4.938354   0.000916   0.000305   0.016174    0.0004
         5         4.936829   0.000610   0.000000   0.014648    0.0005
         6         4.932861   0.000305   0.000610   0.018005    0.0006
         7         4.934387  -0.000305   0.000305   0.024719    0.0007
         8         4.933777  -0.000305   0.000305   0.027161    0.0008
         9         4.928589  -0.000610   0.000000   0.023499    0.0009
         10        0.032959  -0.000305   0.000000   0.022278    0.0010
         11        0.032654  -0.000305  -0.000305   0.018005    0.0011
         12        0.032654   0.000305  -0.000305   0.015259    0.0012
         13        0.032654   0.000000  -0.000305   0.024414    0.0013
         14        0.032959   0.000305   0.000000   0.031433    0.0014
         15        0.032654   0.000305  -0.000305   0.033264    0.0015
         16        0.032654  -0.000610  -0.000610   0.036316    0.0016
         17        0.032959  -0.000305  -0.000305   0.034180    0.0017
         18        0.032043  -0.000610   0.000305   0.028381    0.0018
         19        0.032043  -0.000916  -0.000610   0.022583    0.0019
         20        0.031433  -0.000610   0.000000   0.024414    0.0020
         21        0.030518  -0.000610  -0.000305   0.032959    0.0021
         22        0.030212   0.000305   0.000305   0.032654    0.0022
         23        0.029602   0.000305   0.000305   0.031433    0.0023
         24        0.029907   0.000610   0.000305   0.027161    0.0024
         25        0.029297   0.000610   0.000305   0.020752    0.0025
         26        0.029297   0.000916   0.000000   0.018921    0.0026
         27        0.028992   0.000305   0.000305   0.017395    0.0027
         28        0.030212  -0.000305   0.000000   0.023499    0.0028
         29        0.029297  -0.000305   0.000000   0.026550    0.0029
...                     ...        ...        ...        ...       ...
sweep018 1029970   0.028687  -0.000305  -0.000305   0.035400  102.9970
         1029971   0.028687  -0.000610   0.000305   0.037537  102.9971
         1029972   0.028992   0.000305  -0.000305   0.029297  102.9972
         1029973   0.029297   0.000305  -0.000305   0.021362  102.9973
         1029974   0.029297   0.000916   0.000305   0.020142  102.9974
         1029975   0.029297   0.000916   0.000305   0.020447  102.9975
         1029976   0.029602  -0.000610   0.000000   0.024719  102.9976
         1029977   0.029602  -0.000610  -0.000305   0.027771  102.9977
         1029978   0.028992   0.000000   0.000305   0.030823  102.9978
         1029979   0.029602  -0.000916   0.000916   0.026245  102.9979
         1029980   0.029602  -0.000305   0.000305   0.021057  102.9980
         1029981   0.029907  -0.000305   0.000000   0.020752  102.9981
         1029982   0.029297   0.000610  -0.000305   0.020142  102.9982
         1029983   0.029602   0.000305  -0.000305   0.020752  102.9983
         1029984   0.028992   0.000000   0.000000   0.028687  102.9984
         1029985   0.028687   0.000610   0.000610   0.029907  102.9985
         1029986   0.028381   0.000000   0.000305   0.028076  102.9986
         1029987   0.029602  -0.000610   0.000000   0.018311  102.9987
         1029988   0.028687  -0.000610   0.000000   0.011597  102.9988
         1029989   0.029297  -0.000305   0.000305   0.014648  102.9989
         1029990   0.028992  -0.000305   0.000000   0.021362  102.9990
         1029991   0.028992   0.000305   0.000000   0.028381  102.9991
         1029992   0.028992   0.000610   0.000000   0.030823  102.9992
         1029993   0.028992   0.000610   0

In [7]:
# plot sweeps (class_name, df, list of sweeps to load or 'all', list of channels, time to window, 
# borderspace, headspace, column name to use for x axis, y axis label, inline plotting=True/False)
# use _recording.cont_df for rotary encoder data _recording.class_df if none 
plot_multiple_sweeps_from_class(_recording, _recording.class_df, ['sweep001']
                                , ['channel_0', 'channel_1','channel_2','channel_3'], (0,0), 0.1,
                               0.1, 'time', 'time (seconds)', 'detector power', False)

/Users/johnmarshall/miniconda2/envs/py36_photometry_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:28: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/Users/johnmarshall/miniconda2/envs/py36_photometry_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:29: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


()

In [8]:
#save df to .hdf5 file
fname_for_saving = file.rstrip('.abf')
_recording.save_to_hdf(fname_for_saving) 

True

# load demodulation parameters from file and run demodulation

In [13]:
# default parameters are stored in 'default_photometry_params.yml' in photometry_3.0 folder 
with open('default_photometry_params.yml', 'r') as stream:
    params_for_demodulation_file = yaml.load(stream)

In [18]:
# use _recording.cont_df for rotary encoder data _recording.class_df if none 
_recording_demod = _recording.demodulate_file(_recording.class_df, params_for_demodulation_file)

demodulating sweep sweep001
prefilter channel 1


/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/photometry_analysis/photometry_3.0_module/photometry_3.py:486: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  sweep_dict['time'] = file_df.loc[sweep]['time'].as_matrix()[windowed_start:windowed_end]
/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/photometry_analysis/photometry_3.0_module/photometry_3.py:489: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  encoder_windowed = file_df.loc[sweep][encoder_channel].as_matrix()[windowed_start:windowed_end]
/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/photometry_analysis/photometry_3.0_module/photometry_3.py:493: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  fluorescence_channel_1_windowed = file_df.loc[sweep][fluorescence_channel_1].as_matrix()[windowed_start:windowed_end]
/Users/johnmarshall/Documents/Analysis/PythonA

demodulating channel 1
demodulating/summing in quadrature channel 1
prefilter channel 2
demodulating channel 2
demodulating/summing in quadrature channel 2
sweep sweep001 done
demodulating sweep sweep002
prefilter channel 1
demodulating channel 1
demodulating/summing in quadrature channel 1
prefilter channel 2
demodulating channel 2
demodulating/summing in quadrature channel 2
sweep sweep002 done
demodulating sweep sweep003
prefilter channel 1
demodulating channel 1
demodulating/summing in quadrature channel 1
prefilter channel 2
demodulating channel 2
demodulating/summing in quadrature channel 2
sweep sweep003 done
demodulating sweep sweep004
prefilter channel 1
demodulating channel 1
demodulating/summing in quadrature channel 1
prefilter channel 2
demodulating channel 2
demodulating/summing in quadrature channel 2
sweep sweep004 done
demodulating sweep sweep005
prefilter channel 1
demodulating channel 1
demodulating/summing in quadrature channel 1
prefilter channel 2
demodulating cha

In [22]:
plot_multiple_sweeps_from_class(_recording_demod, _recording_demod.class_df, ['sweep001'],
                                ['encoder_windowed', 'channel_1_demod', 'channel_2_demod'], (5,5), 0.1,
                               0.1, 'time', 'time (seconds)', 'detector power', False)

/Users/johnmarshall/miniconda2/envs/py36_photometry_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:28: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/Users/johnmarshall/miniconda2/envs/py36_photometry_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:29: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


()

In [18]:
#save df to .hdf5 file
_recording_demod.save_to_hdf(fname_for_saving) 

True

# functions to calculate baseline/variance/smooth filter a given channel 

In [19]:
#_recording_demod  = photometry_3.photometry_df_from_file('/Users/johnmarshall/Documents/Analysis/PhotometryData/October2018/10262018/18o26000demodulated.h5')
#fname_for_saving = '/Users/johnmarshall/Documents/Analysis/PhotometryData/October2018/10262018/18o26000'

In [20]:
class_name = _recording_demod
df_input = _recording_demod.class_df
channels_to_calc = ['channel_1_demod', 'channel_2_demod']
window_time = (5,5)
sliding_baseline_length = 2

In [21]:
_recording_demod_F_F0 = photometry_3.transform_df(class_name, 
                                           channels_to_calc, 
                                           window_time, 
                                           {'F_F0': sliding_baseline_length})

/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/photometry_analysis/photometry_3.0_module/photometry_3.py:600: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  sweep_dict[column_nv] = df_input.loc[sweep][column_nv].as_matrix()[windowed_start:windowed_end]
/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/photometry_analysis/photometry_3.0_module/photometry_3.py:603: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  array_length = len(df_input.loc[sweep]['time'].as_matrix()[windowed_start:windowed_end])
/Users/johnmarshall/Documents/Analysis/PythonAnalysisScripts/photometry_analysis/photometry_3.0_module/photometry_3.py:629: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  toadd = F_f0(df_input.loc[sweep][column_v].as_matrix()[windowed_start:windowed_end], Fs, sliding_baseline_length)


In [22]:
#save df to .hdf5 file
#fname_for_saving = '/Users/johnmarshall/Documents/Analysis/PhotometryData/August2018/08282018/18828000_F_F0.h5'
_recording_demod_F_F0.save_to_hdf(fname_for_saving+'_F_F0') 

True

In [23]:
_recording_demod_F_F0.class_df.columns.values

array(['time', 'encoder_windowed', 'fluorescence_channel_1_windowed',
       'channel_1_demod', 'channel_2_demod', 'channel_1_demod_F_F0',
       'channel_2_demod_F_F0'], dtype=object)

In [29]:
plot_multiple_sweeps_from_class(_recording_demod_F_F0, _recording_demod_F_F0.class_df, ['sweep005'],
                                ['encoder_windowed','channel_1_demod_F_F0', 'channel_2_demod_F_F0'], (0,0), 0.1,
                               0.1, 'time', 'time (seconds)', 'detector power', False)

/Users/johnmarshall/miniconda2/envs/py36_photometry_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:28: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/Users/johnmarshall/miniconda2/envs/py36_photometry_analysis/lib/python3.6/site-packages/ipykernel_launcher.py:29: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


()

In [ ]:
class_name = _recording_demod_F_F0
channels_to_calc = ['encoder_windowed']
window_time = (0,0)

In [ ]:
_recording_demod_F_F0_endcodervariance = photometry_3.transform_df(class_name, 
                                           channels_to_calc, 
                                           window_time, 
                                           {'variance': 0})

In [ ]:
_recording_demod_F_F0_endcodervariance.class_df.columns.values

In [ ]:
_recording_demod_F_F0_endcodervariance.class_df.to_hdf('/Users/johnmarshall/Documents/Analysis/PhotometryData/August2018/08102018/18810002_AAV29_002_treadmill_F_F0.h5', key='df', mode='w')

In [ ]:
#save df to .hdf5 file
fname_for_saving = '/Users/johnmarshall/Documents/Analysis/PhotometryData/August2018/08132018/18813000_AAV29_003_treadmilldemodulated_encodervariance'
_recording_demod_F_F0_endcodervariance.save_to_hdf(fname_for_saving) 

In [ ]:
plot_multiple_sweeps_from_class(_recording_demod_F_F0_endcodervariance, 
                                _recording_demod_F_F0_endcodervariance.class_df, ['sweep004'],
                                ['encoder_windowed_variance','encoder_windowed', 'channel_1_demod_F_F0'], 
                                (10,10), 0.1,
                               0.1, 'time', 'time (seconds)', 'detector power', False)

# functions to align sweeps to threshold events 

In [ ]:
## need to trigger files 


def return_indicies(class_name, channel_to_trigger, amplitude_threshold, time_threshold, window_time):

    """
    Input: 
            
            window time in tuple as before    
    
    Output: numpy array of sample points from input channel where trace passes above threshold
    """
    indicies_from_trigger_channel_by_sweep = {}
    for sweep in sorted(list(set(class_name.class_df.index.get_level_values(0)))):
        trace = class_name.class_df.loc[sweep][channel_to_trigger].values
        event_number, sample_indicies = photometry_3.count_events_in_array(trace,
                                                                 class_name.dt, 
                                                                 time_threshold, 
                                                                 amplitude_threshold)
        # remove sample indicies outside of "window"
        sample_indicies_to_add = [i for i in sample_indicies if (window_time[0] < i < window_time[1])]
        
        if event_number > 0:
            indicies_from_trigger_channel_by_sweep[sweep] = sample_indicies_to_add

    return(indicies_from_trigger_channel_by_sweep)

def align_on_trigger(class_name, channel_to_trigger, channels_to_align, amplitdue_threshold, time_threshold, 
                     time_region, window_time=(0,0)):
    """
    Output: dataframe of trace regions surrounding threshold for all channels
    """
    
    # call return indicies to get sample indicies
    
    indicies_from_trigger_channel = return_indicies(class_name, channel_to_trigger, 
                                                    amplitdue_threshold, time_threshold, window_time)
    
    by_channel_by_sweep = {}
    region_in_samples = time_region*class_name.Fs
    
    channels_to_align.append(channel_to_trigger)
    for channel in channels_to_align:
        for sweep in sorted(list(set(class_name.class_df.index.get_level_values(0)))):
            if sweep in list(indicies_from_trigger_channel.keys()):
                
                by_index = {}
                for i in indicies_from_trigger_channel[sweep]:
            
                    
                    if int(i-region_in_samples) < 0:
                        trace_region = np.pad(class_name.class_df.loc[sweep][channel].values[0:int(i+region_in_samples)], 
                                              (int(region_in_samples*2)-int(i+region_in_samples), 0), 'constant', constant_values=np.nan)
                    
                    
                    elif int(i+region_in_samples) > len(class_name.class_df.loc[sweep][channel].values):
                        trace_region = np.pad(class_name.class_df.loc[sweep][channel].values[(i-region_in_samples):], 
                                              (0, int(i+region_in_samples)-len(class_name.class_df.loc[sweep][channel].values))
                                              , 'constant', constant_values=np.nan)
                        
                    else:
                        trace_region = class_name.class_df.loc[sweep][channel].values[int(i-region_in_samples):int(i+region_in_samples)]

                    by_index[i] = trace_region
                    
                index = list(range(int(-region_in_samples), int(region_in_samples)))
        
                by_index = pd.DataFrame(by_index)
                                                                
                by_channel_by_sweep[(channel, sweep)] = by_index
            else:
                pass
             
    return(by_channel_by_sweep)
        
        
def average_sweep_responses(dict_of_dfs):
    
    channels = list(set((list(regions.keys()))[i][0] for i in range(len(regions.keys()))))
    sweeps = list(set((list(regions.keys()))[i][1] for i in range(len(regions.keys()))))
    
    channel_means = {}
    
    for channel in channels:
        list_to_concat = []
        for sweep in sweeps:
            for key in list(dict_of_dfs.keys()):
                if channel in key:
                    list_to_concat.append(regions[key].mean(axis=1))
        
        channel_means[channel] = pd.concat(list_to_concat, axis=1).mean(axis=1)
        
    mean_df = pd.DataFrame(channel_means)
    
    return(mean_df)
        
    

In [ ]:
_recording_demod_F_F0_endcodervariance.class_df['encoder_windowed_variance'].mean()

In [ ]:
_recording_demod_F_F0_endcodervariance.class_df['encoder_windowed_variance'].std()

In [ ]:
np.mean(_recording_demod_F_F0_endcodervariance.class_df['encoder_windowed_variance'].values)

In [ ]:
np.std(_recording_demod_F_F0_endcodervariance.class_df['encoder_windowed_variance'].values)

In [ ]:
regions = align_on_trigger(_recording_demod_F_F0_endcodervariance, 
                                                     'encoder_windowed_variance', 
                                                     ['channel_1_demod_F_F0', 'channel_2_demod_F_F0'],
                                                     8, 
                                                     1000, 
                                                     5)

In [ ]:
regions[('encoder_windowed_variance', 'sweep013')]

In [ ]:
mean_df = average_sweep_responses(regions)

In [ ]:
list(regions.keys())

In [ ]:
plt.subplot(3,1,1)
plt.plot(mean_df['encoder_windowed_variance'].values)
plt.xlabel('time (samples (10000Hz))')
plt.ylabel('encoder variance')
plt.subplot(3,1,2)
plt.plot(mean_df['channel_1_demod_F_F0'].values)
plt.xlabel('time (samples (10000Hz))')
plt.ylabel('F/F0 488')
plt.subplot(3,1,3)
plt.plot(mean_df['channel_2_demod_F_F0'].values)
plt.xlabel('time (samples (10000Hz))')
plt.ylabel('F/F0 405')
plt.show()

In [ ]:
mean_df